In [22]:
class Album():
    def __init__(self, name, artist, songs, genre) -> None:
        self.name = name
        self.artist = artist
        self.songs = songs
        self.genre = genre
        
    def add_song(self, song):
        self.songs.append(song)
        
        
    def remove_song(self, song):
        self.songs.remove(song)
        
    
    def __str__(self) -> str:
        return f"Album {self.name} by {self.artist} in {self.genre}\nTracklist:\n {self.songs}"
        
    
        
        

In [24]:
al1 = Album("the last one","Linkin Park", "In the end - Resynced", "Rock")

In [35]:
print(al1)

Album the last one by Linkin Park in Rock
Tracklist:
 In the end - Resynced


## Open / Closed Principle ## 

Software entities (classes, modules, functions, etc.) should be open for extension, but closed for modification.

This means that I should be able to add new functionality without changing my existing code structure, but by adding new code instead. 

In [26]:
class SearchBy:
    def is_matched(self, album):
        pass
    
    def __and__(self, other):
        return AndSearchBy(self, other)

    
class AndSearchBy(SearchBy):
    def __init__(self, SearchBy1, SearchBy2):
        self.SearchBy1= SearchBy1
        self.SearchBy2 = SearchBy2
        
    def is_matched(self, album):
        return self.SearchBy1.is_matched(album)and self.SearchBy2.is_matched(album)
        
        
class SearchByGenre(SearchBy):
    def __init__(self, genre):
        self.genre = genre
        
    def is_matched(self, album):
        return album.genre == self.genre
    
class SearchByArtist(SearchBy):
    def __init__(self, artist):
        self.artist = artist
    def is_matched(self, album):
        return album.artist == self.artist
    
class AlbumBrowser:
    def browse(self, albums, searchby):
        return [album for album in albums if searchby.is_matched(album)]

In [33]:
LAWoman = Album(
    name="L.A. Woman",
    artist="The Doors",
    songs=["Riders on the Storm"],
    genre="Rock",
)
Trash = Album(
    name="Trash",
    artist="Alice Cooper",
    songs=["Poison"],
    genre="Rock",
)
albums = [LAWoman, Trash, al1]
# this creates the AndSearchBy object
my_search_criteria = SearchByGenre(genre="Rock") & SearchByArtist(
    artist="The Doors"
)
browser = AlbumBrowser()
assert browser.browse(albums=albums, searchby=my_search_criteria) == [LAWoman]
# yay we found our album

print(browser.browse(albums=albums, searchby=my_search_criteria)[0])
        

Album L.A. Woman by The Doors in Rock
Tracklist:
 ['Riders on the Storm']


## Liskov Substitution Principle ##

The principle defines that objects of a superclass shall be replaceable with objects of its subclasses without breaking the application. That requires the objects of your subclasses to behave in the same way as the objects of your superclass.

This means that if we have a base class T and subclass S, you should be able to substitute the main class T with the subclass S without breaking the code. The interface of a subclass should be the same as the interface of the base class, and the subclass should behave in the same way as the base class.

In the classic example with rectangles and squares, we create a Rectangle class, with width and height setters. If you have a square, the width setter also needs to resize the height, and vice versa to keep the square property. This forces us to make a choice: we either keep the implementation of the Rectangle class, but then Square stops being a square when you use the setter on it, or you change the setters to make height and width the same for squares. This could lead to some unexpected behaviour if you have a function that resizes the height of your shape.

In [42]:
class Rectangle:
    def __init__(self, height, width):
        self._height = height
        self._width = width
        
    @property    
    def width(self):
        return  self._width 
    
    @width.setter
    def width(self, value):
        self._width = value

    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, value):
        self._height = value
        
    def get_area(self):
        return self._height * self._width
    
    
    
        
class Square(Rectangle):
    def __init__(self, size):
        Rectangle.__init__(self, size, size)
        
    @Rectangle.height.setter
    def height(self, size):
        self._height = size
        self._width = size
        
    @Rectangle.width.setter
    def width(self, size):
        self._width = size
        self._height = size
        
    

In [55]:
def get_squashed_height_area(Rectangle):
    #Rectangle.height = 1
    area = Rectangle.get_area()
    return area

rectangle = Rectangle(5, 5)
square = Square(3)

print(get_squashed_height_area(rectangle))  ## 5
print(get_squashed_height_area(square)) ## 1


25
9


## Interface Segregation Principle ##

“Clients should not be forced to depend upon interfaces that they do not use.”

If you have a base class with many methods, possibly not all of your subclasses are going to need them, maybe just a few. But due to inheritance, you will be able to call these methods on all the subclasses, even on those that don’t need it. This means a lot of interfaces that are unused, unneeded and will result in bugs when they get accidentally called.

This principle is meant to prevent this from happening. We should make interfaces as small as possible, so that we don’t need to implement functions we don’t need. Instead of one big base class, we should split them into multiple ones. They should only have methods that make sense for each, and then have our subclasses inherit from them.



In [46]:
class PlaySongs:
    def __init__(self, title):
        self.title = title
    def play_drums(self):
        print("Ba-dum ts")
    def play_guitar(self):
        print("*Soul-moving guitar solo*")
    def sing_lyrics(self):
        print("NaNaNaNa")
# This class is fine, just changing the guitar and lyrics
class PlayRockSongs(PlaySongs): 
    def play_guitar(self):
        print("*Very metal guitar solo*")
    def sing_lyrics(self):
        print("I wanna rock and roll all night")
# This breaks the ISP, we don't have lyrics 
class PlayInstrumentalSongs(PlaySongs):
    def sing_lyrics(self):
        raise Exception("No lyrics for instrumental songs")

Instead, we could have a class for the singing and the music separately (assuming guitar and drums always happen together in our case, otherwise we need to split them up even more, perhaps by instrument.) This way, we only have the interfaces we need, we cannot call sing lyrics on instrumental songs.

In [51]:
from abc import ABCMeta, abstractmethod

class PlaySongsLyrics:
    @abstractmethod
    def sing_lyrics(self, title):
        pass
class PlaySongsMusic:
    @abstractmethod
    def play_guitar(self, title):
        pass
    @abstractmethod
    def play_drums(self, title):
        pass
class PlayInstrumentalSong(PlaySongsMusic):
    def play_drums(self, title):
        print("Ba-dum ts")
    def play_guitar(self, title):
        print("*Soul-moving guitar solo*")
class PlayRockSong(PlaySongsMusic, PlaySongsLyrics):
    def play_guitar(self):
        print("*Very metal guitar solo*")
    def sing_lyrics(self):
        print("I wanna rock and roll all night")
    def play_drums(self, title):
        print("Ba-dum ts")

## Dependency Inversion Principle  ##

High-level modules should not depend on low-level modules. Both should depend on abstractions (e.g. interfaces).
Abstractions should not depend on details. Details (concrete implementations) should depend on abstractions

If your code has well-defined abstract interfaces, changing the internal implementation of one class shouldn’t break your code. A class it interacts with should not have knowledge of the inner workings of the other class, and should be unaffected as long as the interfaces are the same. An example would be changing the type of database you use (SQL or NoSQL) or changing the data structure you store your data in (dictionary or list).


This is illustrated in the following example, where ViewRockAlbums explicitly depends on the fact that albums are stored in a tuple in a certain order inside AlbumStore. It should have no knowledge of the internal structure of Albumstore. Now if we change the ordering in the tuples in the album, our code would break.

In [52]:
class AlbumStore:
    albums = []
    def add_album(self, name, artist, genre):
        self.albums.append((name, artist, genre))
class ViewRockAlbums:
    def __init__(self, album_store):
        for album in album_store.albums:
            if album[2] == "Rock":
                print(f"We have {album[0]} in store.")

Instead, we need to add an abstract interface to AlbumStore to hide the details, that can be called by other classes. This should be done as in the example in the Open-Closed Principle, but assuming we don’t care about filtering by anything else, I’ll just add a filter_by_genre method. Now if we had another type of AlbumStore, that decides to store the album differently, it would need to implement the same interface for filter_by_genre to make ViewRockAlbums work.

In [53]:
class GeneralAlbumStore:
    @abstractmethod
    def filter_by_genre(self, genre):
        pass
class MyAlbumStore(GeneralAlbumStore):
    albums = []
    def add_album(self, name, artist, genre):
        self.albums.append((name, artist, genre))
    def filter_by_genre(self, genre):
        if album[2] == genre:
            yield album[0]
class ViewRockAlbums:
    def __init__(self, album_store):
        for album_name in album_store.filter_by_genre("Rock"):
            print(f"We have {album_name} in store.")